In [12]:
import os
import torch


paramdir = "./all_outputs/"
dirlist = os.listdir(paramdir)
print("Total:", len(dirlist), "files")
mask_token = 65000
max_seq_length = 512


def TaskEncode(l_list):
    global URLs, TASKTYPEs
    return int(l_list[0]) + int(l_list[1]) * len(URLs) + int(l_list[2]) * (len(URLs) * len(TASKTYPEs))

def TaskDecode(_num):
    global URLs, TASKTYPEs
    url_id = _num % len(URLs)
    met_id = _num // (len(TASKTYPEs) * len(URLs))
    tsk_id = (_num - url_id - met_id * (len(TASKTYPEs) * len(URLs))) // len(URLs)
    return [url_id, tsk_id, met_id]


def get_proc(fname):
    proc_list = []
    with open(fname) as proc:
        for line in proc.readlines():
            if (l := line.strip()) != "":
                l_list = l.split(", ")
                #proc_list.append(l_list)
                proc_list.append(TaskEncode(l_list))
    return proc_list


def DatasetGen(dset, batch_size):
    def align(lst):
        lst.extend((0 for _ in range(max_seq_length - len(lst))))
        return lst

    global mask_token, paramdir

    res = [[], []]
    for f in dset:
        ind = 0
        proc = get_proc(paramdir + f)
        tmp2 = [proc[ind]]
        while (ind := ind + 1) < len(proc):
            res[0].append(align([*tmp2, mask_token]))
            tmp2.append(proc[ind])
            res[1].append(align(list(tmp2)))
            if len(res[0]) >= batch_size:
                yield (torch.tensor(res[0], dtype=torch.int), torch.tensor(res[1], dtype=torch.int))
                res = [[], []]

    if len(res[0]):
        yield (torch.tensor(res[0], dtype=torch.int), torch.tensor(res[1], dtype=torch.int))

def DatasetGenWholeProc(dset, batch_size):
    def align(lst: list) -> list:
        lst.extend((0 for _ in range(max_seq_length - len(lst))))
        return lst

    tmp1, tmp2, tmp3 = [], [], []
    global paramdir
    for f in dset:
        proc = get_proc(paramdir + f)
        tmp1.append(align(proc))
        tmp2.append(align(proc[:-1]))
        tmp3.append(align(proc[1:]))
        if len(tmp1) >= batch_size:
            yield torch.tensor(tmp1, dtype=torch.long), torch.tensor(tmp2, dtype=torch.long), torch.tensor(tmp3, dtype=torch.long)
            tmp1, tmp2, tmp3 = [], [], []
    if len(tmp1):
        yield torch.tensor(tmp1, dtype=torch.long), torch.tensor(tmp2, dtype=torch.long), torch.tensor(tmp3, dtype=torch.long)


# Get parameter options: all_outputs/*.txt files contain indices
def get_options(fname):
    _URLs, _TASKTYPEs, _METHODs, tmp = [], [], [], []

    with open(fname) as params:
        lines = params.readlines()
        for string in lines:
            string = string.strip()
            if string == "URLs:":
                tmp = _URLs
                continue
            if string == "Task types:":
                tmp = _TASKTYPEs
                continue
            if string == "Methods:":
                tmp = _METHODs
                continue
            tmp.append(string)
    return _URLs, _TASKTYPEs, _METHODs

URLs, TASKTYPEs, METHODs = get_options("./unique_params.txt")
mask_token = len(URLs) * len(TASKTYPEs) * len(METHODs) + 4
print("Total:", len(URLs), "URLs,", len(TASKTYPEs), "task types,", len(METHODs), "methods")

Total: 39327 files
Total: 285 URLs, 13 task types, 4 methods


In [13]:
import random as rng


rng.seed(4815162342)
rng.shuffle(dirlist)
train, val, test = dirlist[: int(0.8 * len(dirlist))], dirlist[int(0.8 * len(dirlist)) : int(0.9 * len(dirlist))], dirlist[int(0.9 * len(dirlist)) :]

In [14]:
import torch.nn as nn
import torch.optim as optim
import math


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output


class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x


class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x


class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [15]:
# Generate random sample data
# src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [21]:
src_vocab_size = len(URLs) * len(TASKTYPEs) * len(METHODs) + 5
tgt_vocab_size = len(URLs) * len(TASKTYPEs) * len(METHODs) + 5
num_layers = 6
num_heads = 8
d_model = 512
dropout = 0.1
d_ff = 2048

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr= 1e-4, betas= (0.9, 0.98), eps= 1e-9)

transformer.train()
for epoch in range(100):
    i = 1
    for source, t_target, l_target in DatasetGenWholeProc(train, 4):
        optimizer.zero_grad()
        output = transformer(source, t_target)
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), l_target.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        print("Iteration", i, f": loss = {loss.item()}", end= '\r')
        i += 1
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

KeyboardInterrupt: 

In [23]:
# up to 2462, start from 2463
torch.save(transformer, "./tformer2463.ckpt")

In [24]:
torch.load("./tformer2463.ckpt")

Transformer(
  (encoder_embedding): Embedding(14825, 512)
  (decoder_embedding): Embedding(14825, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLayer(